In [1]:
%matplotlib notebook
import numpy as np
import pandas
import matplotlib.pyplot as plt
import os

Let's import the SED fitting code which use prospector package.

In [2]:
from pyprosp import prospector

# Data

For this tutorial, we will fit the SED based on some photometric measurements.<br>
What's important with input data is the names you give:

- measurements must be on the format "instrument.band" (e.g. sdss.u, ps1.g, ...)
- uncertainties must be on the format "instrument.band.err" (e.g. sdss.u.err, ps1.g.err, ...)

The measurement uncertainties are necessary for the code to automatically recognize the filters used.<br>
Here we set a pandas.Series with data, as it could come from a DataFrame in a loop, but it also works with a dictionary.

These data are coming from "Johnson et al. 2013", the studied galaxy is "NGC4163", measured in maggies.

In [3]:
data = {"SN_name":"NGC4163",
        "zcmb":0.000551,
        "GALEX.FUV":5.91561634e-08, 
        "GALEX.FUV.err":5.91561634e-09, 
        "GALEX.NUV":8.87156012e-08, 
        "GALEX.NUV.err":8.87156012e-09, 
        "sdss.u":2.22843515e-07,
        "sdss.u.err":2.22843515e-08,
        "sdss.g":4.87528490e-07,
        "sdss.g.err":4.87528490e-08,
        "sdss.r":6.98232404e-07,
        "sdss.r.err":6.98232404e-08,
        "sdss.i":8.79022517e-07,
        "sdss.i.err":8.79022517e-08,
        "sdss.z":9.46237161e-07,
        "sdss.z.err":9.46237161e-08, 
        "Spitzer.1":4.32513831e-07, 
        "Spitzer.1.err":4.32513831e-08, 
        "Spitzer.2":2.80543364e-07, 
        "Spitzer.2.err":2.80543364e-08, 
        "Spitzer.3":1.95884467e-07, 
        "Spitzer.3.err":1.95884467e-08, 
        "Spitzer.4":1.29419584e-07, 
        "Spitzer.4.err":1.29419584e-08, 
        }

data = pandas.Series(data)
data

SN_name              NGC4163
zcmb                0.000551
GALEX.FUV        5.91562e-08
GALEX.FUV.err    5.91562e-09
GALEX.NUV        8.87156e-08
GALEX.NUV.err    8.87156e-09
sdss.u           2.22844e-07
sdss.u.err       2.22844e-08
sdss.g           4.87528e-07
sdss.g.err       4.87528e-08
sdss.r           6.98232e-07
sdss.r.err       6.98232e-08
sdss.i           8.79023e-07
sdss.i.err       8.79023e-08
sdss.z           9.46237e-07
sdss.z.err       9.46237e-08
Spitzer.1        4.32514e-07
Spitzer.1.err    4.32514e-08
Spitzer.2        2.80543e-07
Spitzer.2.err    2.80543e-08
Spitzer.3        1.95884e-07
Spitzer.3.err    1.95884e-08
Spitzer.4         1.2942e-07
Spitzer.4.err     1.2942e-08
dtype: object

# Prospector SED fitting

### Initialization

Now that we have data to work with, let's build a prospector object.

In [5]:
prosp = prospector.Prospector()

Prospector can work either with measured photometry or spectrum (or with both).<br>
Here we give the data we constructed above (`phot=data`). Note that no matter there are some additional informations in these data, the code ignore them, only photometry is managed (with the constraints on name format highlighted above).<br>
We let `spec` to None (`spec=None`, default value).
We inform the code of the measurement unit (`unit="mgy"`). Note that there are other available units treated by the code:
- "Hz": erg/s/cm2/Hz
- "AA": erg/s/cm2/AA
- "mgy": maggies
- "Jy": Jansky
- "mag": magnitudes

This unit defines the your measurements in general, whether you give photometry or spectrometry. However if you give both, you cannot define a different unit for each one. They must be in the same unit.

The builder also accept a redshift value to fix it in the SED fitting. If `z` is set to None, the redshift becomes a free parameter for the fit.<br>
Finally, you can optionally give an object name.

In [9]:
prosp.set_data(phot=data, spec=None, unit="mgy", z=data.zcmb, name=data.SN_name)

# Can be executed directly in the builder:
# prosp = prospector.Prospector(phot=data, spec=None, unit="mgy", z=data.zcmb, name=data.SN_name)

Prospector needs now three specific objects to be built: 
- the `obs` containing every measurements
- the `model` synthesizing the parameters defining the SED model. The free paramters of the model will be fitted during SED fitting.
- the `sps` refering to the Stellar Population Synthesis, calling FSPS procedures.

It doesn't matter in which order you build `obs` and `model`, but `sps` has to be built after the `model` (because it depends on the paramters implemented in the `model`).

### Build `obs`

Firstly, let's build the `obs` object, which basically is a dictionary formatted to be used in prospector.

In this example, we are working with photometry. We may want to avoid some filter measurements during the SED fitting. This can be done by giving a list of filters to be used for the fit to `phot_mask` argument (there is an example in next cell in comment). Filters must be on the format "instrument.band" (e.g. sdss.u, ps1.g, ...).

A few options allow to print some user messages:
- `verbose`: print the built `obs` dictionary (default is False)
- `warning`: print warning messages if there's any, noticing some anomalies but without breaking the code

In addition of this example, if you are working on spectroscopy, you can also apply a mask on the measured spectrum by giving limits to `spec_mask` argument in Angstroms (a few examples below).

Finally, if you already have an `obs` prospector compatible dictionary, you can give it in `obs` argument. In this case, `set_data` allow you to choose if you want to save the data from this dictionary in your instance's attributes.

In [8]:
# Example of photometric mask:
# phot_mask = ["sdss.u", "sdss.g", "sdss.r", "sdss.i", "sdss.z"]

# Examples of spectroscopic masks:
# spec_mask = None ; (None, None) ; (1000, None) ; (1e3, 1e5) ; ...

prosp.build_obs(phot_mask=None, verbose=True, warnings=True)#, spec_mask=(None, None), obs=None, set_data=False)


#===============#
#   Built obs   #
#===============#

{'filternames': ['galex_FUV',
                 'galex_NUV',
                 'sdss_u0',
                 'sdss_g0',
                 'sdss_r0',
                 'sdss_i0',
                 'sdss_z0',
                 'spitzer_irac_ch1',
                 'spitzer_irac_ch2',
                 'spitzer_irac_ch3',
                 'spitzer_irac_ch4'],
 'filters': [<class 'sedpy.observate.Filter'>(galex_FUV),
             <class 'sedpy.observate.Filter'>(galex_NUV),
             <class 'sedpy.observate.Filter'>(sdss_u0),
             <class 'sedpy.observate.Filter'>(sdss_g0),
             <class 'sedpy.observate.Filter'>(sdss_r0),
             <class 'sedpy.observate.Filter'>(sdss_i0),
             <class 'sedpy.observate.Filter'>(sdss_z0),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch1),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch2),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch3),
    

### Build 'model'

This is the trickiest step of the initialization as there are a lot of possibilities highly impacting the SED fitting.


In [ ]:
model_tuto = {"mass_tuto":{"N": 1,
                           "isfree": True,
                           "init": 1e8,
                           "prior": dict(name="LogUniform", mini=1e6, maxi=1e12),
                           "init_disp": 1e6, 
                           "disp_floor": 1e6, 
                           "units": "solar masses formed"}}

In [ ]:
prosp.build_model(model=model_tuto, templates=["parametric_sfh", "dust_emission"], verbose=True, describe=True)